# Preamble

In [1]:
from IPython.display import display, HTML
import json
import collections
import bz2

# Load Data

In [2]:
PATH_CAUSENET = "../../data/causality-graphs/causenet-full.jsonl.bz2"
PATH_RANDOM_NUMBERS = "../../data/random/generated_random_numbers.bz2"

# Load Causality Graphs

In [3]:
def load_jsonl(path):
    print("Loading... " + path)
    lines = []
    document = bz2.open(path, mode='rt')
    for line in document:
        lines.append(json.loads(line))
    return lines

In [ ]:
causenet = load_jsonl(PATH_CAUSENET)

Loading... ../../data/causality-graphs/causenet-full.jsonl.bz2


## Determine Subgraphs

In [5]:
def graph_with_support_values(causality_graphs, graph_type):
    for relation in causality_graphs[graph_type]:
        patterns = []
        for source in relation['sources']:
            is_full_graph = 'causenet' not in graph_type[0]
            if is_full_graph and source['type'] != graph_type[0]:
                continue
            if 'path_pattern' in source['payload']:
                patterns.append(source['payload']['path_pattern'])
        relation['support'] = len(set(patterns))

    return causality_graphs[graph_type]

In [6]:
def belongs_to_high_precision_causenet(sample):
    if sample['support'] > 1:
        return True

    for source in sample['sources']:
        if source['type'] == 'wikipedia_infobox':
            return True

        if source['type'] == 'wikipedia_list':
            return True

    return False

In [7]:
def get_subgraph(graph, source_type):
    subgraph = []
    for relation in graph:
        belongs_to_subgraph = False
        for source in relation['sources']:
            if source['type'] == source_type:
                belongs_to_subgraph = True
                break
        if belongs_to_subgraph:
            subgraph.append(relation)
    return subgraph

In [8]:
source_types = ['clueweb12_sentence', 'wikipedia_sentence',
                'wikipedia_list', 'wikipedia_infobox']
causality_graphs = {('causenet',): causenet, ("causenet_precision",): []}

for relation in graph_with_support_values(causality_graphs, ('causenet',)):
    if belongs_to_high_precision_causenet(relation):
        causality_graphs[("causenet_precision",)].append(relation)

for source_type in source_types:
    causality_graphs[(source_type,)] = get_subgraph(causenet, source_type)

support_levels = [range(1, 2), range(2, 3), range(3, 38)]

for graph_type in [('clueweb12_sentence',), ('wikipedia_sentence',)]:
    for support_level in support_levels:
        graph = graph_with_support_values(causality_graphs, graph_type)
        for relation in graph:
            if relation['support'] in support_level:
                graph_id = (graph_type[0], str(support_level))
                causality_graphs.setdefault(graph_id, []).append(relation)

In [9]:
for graph_type in causality_graphs:
    graph = graph_with_support_values(causality_graphs, graph_type)

    print(graph_type)
    print(f"\t|E|: {len(graph):,}")
    max_support = max([relation['support'] for relation in graph])
    print(f"\tMaximum Support: {max_support:,}")

('causenet',)
	|E|: 11,609,890
	Maximum Support: 38
('causenet_precision',)
	|E|: 197,806
	Maximum Support: 38
('clueweb12_sentence',)
	|E|: 10,872,313
	Maximum Support: 37
('wikipedia_sentence',)
	|E|: 793,593
	Maximum Support: 21
('wikipedia_list',)
	|E|: 10,612
	Maximum Support: 0
('wikipedia_infobox',)
	|E|: 7,880
	Maximum Support: 0
('clueweb12_sentence', 'range(1, 2)')
	|E|: 10,700,845
	Maximum Support: 1
('clueweb12_sentence', 'range(2, 3)')
	|E|: 116,912
	Maximum Support: 2
('clueweb12_sentence', 'range(3, 38)')
	|E|: 54,556
	Maximum Support: 37
('wikipedia_sentence', 'range(1, 2)')
	|E|: 789,381
	Maximum Support: 1
('wikipedia_sentence', 'range(2, 3)')
	|E|: 3,237
	Maximum Support: 2
('wikipedia_sentence', 'range(3, 38)')
	|E|: 975
	Maximum Support: 21


# Manual Evaluation

In [10]:
random_numbers = bz2.BZ2File(PATH_RANDOM_NUMBERS, "r").readlines()
random_numbers = random_numbers[0].decode("utf-8").split(',')
random_numbers = [int(x) for x in random_numbers]

# unique random numbers (without replacement)
random_numbers = list(collections.OrderedDict.fromkeys(random_numbers))
current_random_number_index = 0

In [11]:
def next_random_int():
    global current_random_number_index
    result = random_numbers[current_random_number_index]
    current_random_number_index += 1
    return result

In [12]:
def get_random_subset(full_list, random_subset_length):
    random_number_range = range(random_subset_length + 10)
    random_numbers = [next_random_int() % len(full_list)
                      for _ in random_number_range]
    random_numbers = list(collections.OrderedDict.fromkeys(random_numbers))
    random_numbers = random_numbers[:random_subset_length]
    assert len(random_numbers) == random_subset_length

    random_subset = []
    for random_number in random_numbers:
        random_subset.append(full_list[random_number])
    return random_subset

In [13]:
def pretty_print(string, substring):
    removed_underscores = substring.replace("_", " ")
    bold_string = "<b>" + removed_underscores + "</b>"
    return string.lower().replace(removed_underscores, bold_string)

In [14]:
def random_samples(graph_type, number_samples=100):

    output = ""
    graph = graph_with_support_values(causality_graphs, graph_type)
    random_subset = get_random_subset(graph, number_samples)

    for relation_idx in range(number_samples):
        relation = random_subset[relation_idx]

        output += str(relation_idx + 1) + ". "
        output += "<b>"
        output += relation['causal_relation']['cause']['concept']
        output += "</b> &#8594; "
        output += "<b>"
        output += relation['causal_relation']['effect']['concept'] + "</b>"
        if graph_type[0] not in ['wikipedia_list', 'wikipedia_infobox']:
            output += " (Support: " + str(relation['support']) + ")" + "<br>"

        selected_sources = random_sources(graph_type, relation)
        output += '<table><tr>'
        for source in selected_sources:
            if "</a>" not in source:
                source = pretty_print(
                    source, relation['causal_relation']['cause']['concept'])
                source = pretty_print(
                    source, relation['causal_relation']['effect']['concept'])
            output += '<td style="text-align:left">' + source + "</td>"
        output += "</tr></table><br>"

    display(HTML(output))

In [15]:
WIKIPEDIA_OLD_ID = "https://en.wikipedia.org/w/index.php?&oldid="


def random_sources(graph_type, relation):
    sources = []
    for source in relation['sources']:
        if source['type'] != graph_type[0] and 'causenet' not in graph_type[0]:
            continue

        if source['type'] in ['wikipedia_infobox', 'wikipedia_list']:
            source_description = source['payload']['wikipedia_page_title']
            source_description += ":   "
            source_description += '<a href="' + WIKIPEDIA_OLD_ID
            source_description += source['payload']['wikipedia_revision_id']
            source_description += "\">"
            source_description += WIKIPEDIA_OLD_ID
            source_description += source['payload']['wikipedia_revision_id']
            source_description += "</a>"
            source_description += " (" + source['type'] + ")"
            sources.append(source_description)
        else:
            sentence = source['payload']['sentence']['surface']
            source_description = (sentence, source['type'])
            sources.append(str(source_description))

    sources = list(collections.OrderedDict.fromkeys(sources))

    to_select = 3
    if len(sources) < 3:
        to_select = 1

    selected_sources = []
    unique_sources = 0
    while unique_sources < to_select:
        selected_sources.append(sources[next_random_int() % len(sources)])
        sources_as_dict = collections.OrderedDict.fromkeys(selected_sources)
        unique_sources = len(sources_as_dict)
    selected_sources = list(collections.OrderedDict.fromkeys(selected_sources))

    return selected_sources

## Manual Evaluation of CauseNet

In [16]:
random_samples(graph_type=('causenet',))

"('to summarize, ets exposure is causally associated with cancers of the lung and nasal sinus; the evidence is suggestive of a causal association between ets exposure and cervical cancer.', 'clueweb12_sentence')"
"('its first lead story, “dotygate,” accused duluth mayor gary doty and his administration of various crimes associated with the demolition of buildings on east first street to make way for construction of the duluth technology village.', 'wikipedia_sentence')"
"('the affected sib/relative pair (asp/arp) design is often used with covariates to find genes that can cause a disease in pathways other than through those covariates.', 'clueweb12_sentence')"
"('this civil conflict poisoned the political climate for many years and resulted in instability which, in turn, led to the imposition of a military dictatorship in 1967.', 'clueweb12_sentence')"
"('locula argues that this practice has led to corruption of the political class and manipulation of the electorate.', 'clueweb12_sentence')"
"('frustration with this donor control of the evaluation agenda, together with an early recognition by community groups and community voices that there was an essential role for the community in evaluation, has led to the development of a number of approaches to evaluation based in the community, such as participatory rural appraisal, among others.', 'clueweb12_sentence')"
"('a fuel pump that is simply weak during operation will cause issues with the calibration of your fuel system.', 'clueweb12_sentence')"
"('his besetting faults were a conscious refusal to accept his limitations as a historian, limitations that were inevitable, given his lack of academic training \x97 and the sub-conscious recognition of them that led to an inordinate conceit and to constant belittling of his predecessors.', 'clueweb12_sentence')"
"('the glory mission builds on the heritage of our acrimsat and sorce satellite programs, both of which were launched aboard orbital rockets, and could lead to our support of the future solar monitoring mission as well.""', 'clueweb12_sentence')"
"('lack of purchasing power is leading to unsold inventories and cut-back of production, which is leading to more lay-offs and greater unemployment, resulting in even lesser purchasing power.', 'clueweb12_sentence')"
"('the mri often simply results in unnecessary back surgery (at a cost of over $15,000 — caching!)', 'clueweb12_sentence')"


We consider as incorrect:
- 03, 05, 08, 33, 37, 39, 41, 46, 47, 57,
- 60, 72, 77, 83, 89, 90, 93

$\rightarrow$ estimated precision of $\frac{83}{100}=0.83$

## Manual Evaluation of CauseNet-Precision

In [17]:
random_samples(graph_type=('causenet_precision',))

12, 22, 23, 67

$\rightarrow$ estimated precision of $\frac{96}{100}=0.96$

## Manual Evaluation of ClueWeb-sentence Graph

### Support 1

In [18]:
random_samples(graph_type=('clueweb12_sentence', 'range(1, 2)'))

We consider as incorrect:
- 10, 12, 15, 20, 24, 25, 26, 28, 32, 34,
- 36, 38, 48, 50, 52, 54, 55, 56, 63, 69,
- 70, 73, 75, 86, 88, 90

$\rightarrow$ estimated precision of $\frac{74}{100}=0.74$

### Support 2

In [19]:
random_samples(graph_type=('clueweb12_sentence', 'range(2, 3)'))

We consider as incorrect:
- 1, 33, 46, 55, 64

$\rightarrow$ estimated precision of $\frac{95}{100}=0.95$

### Support 3-37

In [20]:
random_samples(graph_type=('clueweb12_sentence', 'range(3, 38)'))

"('it should also result in conserving water since computerized control of the system allows for watering in the evening and morning hours exclusively, when water loss due to evaporation is minimized.', 'clueweb12_sentence')","('during liquefaction, small volumes of di water were added to compensate for water loss due to evaporation.', 'clueweb12_sentence')","('next, the installation of an irrigation system will help with water loss due to evaporation.', 'clueweb12_sentence')"
"('as well as the documentation, many presentations and workshops were given to schools throughout the province of saskatchewan.concluding comments glasser (1998) emphasized that we need to push harder for ""coercion-free and failure-free quality schools because it is the alienation caused by coercion and punishment that leads young people to turn to drugs.students who have a strong awareness of self, and who have learned and internalized affective skills will be less likely to become behavior problems in school or juvenile delinquents in society.', 'clueweb12_sentence')","('it stresses that rewards can be overused and reduce interest in learning and that punishment can lead to alienation and frustration.', 'clueweb12_sentence')","(""punishment that results in such alienation opposes god's purposes and thereby falls tragically into sin itself."", 'clueweb12_sentence')"
"('this may be a difficult idea to digest, but when we say that ""god does this"" or ""god does that,"" we don\'t believe that god causes things to happen as we do.', 'clueweb12_sentence')","('parsons goes on to describe miracles as the logically possible cases in which god ""might occasionally cause things to happen which otherwise would never occur"".', 'clueweb12_sentence')","('every reasonable man has to admit that god could cause such things to happen; and a christian is obliged to believe that he did so, wherever the bible obviously intends to declare such events to be matters of historical fact.', 'clueweb12_sentence')"
"('the accident scarred up his face and caused him a lot of damage including pain which most likely caused his addiction to pain killers.', 'clueweb12_sentence')","('an official 1963 publication setting forth the united states public health service\'s views made the point clearly: ""last century the pain that led to addiction was often physical; today it is mainly psychic.', 'clueweb12_sentence')","('at clarity way, we offer a specialized treatment plan designed to wean you off the prescription painkillers while at the same time addressing the pain that caused your addiction in the first place.', 'clueweb12_sentence')"
"('no doubt we will see yet another commission recommending the widespread use of so-called antidepressants, never mind these ssris (selective serotonin reuptake inhibitors) are reported to cause “anxiety, agitation, panic attacks, insomnia, irritability, hostility, aggressiveness, impulsivity, akathisia (psychomotor restlessness), hypomania, and mania … in adult and pediatric patients being treated with antidepressants for major depressive disorder as well as for other indications, both psychiatric and nonpsychiatric,” according to david healy and david menkes from cardiff university, and andrew herxheimer from the uk cochrane centre.', 'clueweb12_sentence')","('ssris can cause agitation, nausea, and diarrhea.', 'clueweb12_sentence')","('patient studies suggest that ssris may sometimes cause extreme and unpleasant activation or agitation [42–44], which can resemble neuroleptic-induced akathisia [45].', 'clueweb12_sentence')"
"('yet according to the world health organisation, depression is already the fourth most significant cause of suffering and disease – and by 2020 it will be in second place, behind heart disease.', 'clueweb12_sentence')","('the world health organization claims that depression is the 4th most significant cause of suffering and disability worldwide (after heart disease, cancer, and car accidents) and is expected to reach 2nd place b

We consider as incorrect:
- 20, 71, 94

$\rightarrow$ estimated precision of $\frac{97}{100}=0.97$

## Manual Evaluation of Wikipedia-sentence Graph

### Support 1

In [21]:
random_samples(graph_type=('wikipedia_sentence', 'range(1, 2)'))

"('cause of the failure was a lox valve closing prematurely, which resulted in the rupture of a propellant duct and thrust termination.', 'wikipedia_sentence')"
"('at his request the margrave james brought about the religious disputations of baden, 1589, and emmendingen, 1590.', 'wikipedia_sentence')"
"('m. abscessus can cause lung disease, skin infections, central nervous system infections, bacteremia, eye infections, and other, less common diseases.', 'wikipedia_sentence')"
"('consequently, the country faced a famine that resulted in the death of nearly 1 million people from 1984 to 1986.', 'wikipedia_sentence')"
"('a potassium ion helps tba fold into a g-quadruplex structure, which results in a significant positive band at 295\xa0nm and a negative band at 270\xa0nm on its circular dichroism (cd) spectrum.', 'wikipedia_sentence')"
"('in 1921 gustaaf molengraff, a dutch geologist, postulated that the nearly uniform sea depths of the shelf indicated an ancient peneplain that was the result of repeated flooding events as ice caps melted, with the peneplain becoming more perfect with each successive flooding event.', 'wikipedia_sentence')"
"('the conjunction of the painting and the document led to the identification of the master of the copenhagen charity as bartolomeo ghetti.', 'wikipedia_sentence')"
"('the breakaway is attributed to the marriage of periyar to a lady much younger.', 'wikipedia_sentence')"
"('pleistocene park seeks to test the hypotheses that large herbivores maintained the pleistocene tundra steppe and that overhunting by humans caused both the animals and pleistocene ecosystem to vanish.', 'wikipedia_sentence')"
"(""the kmt's nation-building efforts followed this ideology based on unpleasant experiences with the centrifugal forces during the warlord era in 1916–1928 that had torn the government in china."", 'wikipedia_sentence')"
"('thus the adaaa led to broader coverage of impaired employees.', 'wikipedia_sentence')"


We consider as incorrect:
- 07, 10, 15, 23, 31, 32, 36, 38, 41, 42,
- 45, 52, 55, 66, 69, 74, 76, 79, 87, 90,
- 100

$\rightarrow$ estimated precision of $\frac{79}{100}=0.79$

### Support 2

In [22]:
random_samples(graph_type=('wikipedia_sentence', 'range(2, 3)'))

"('while some of the passengers of the ibis reach their destination in mauritius, others find themselves in hong kong and canton and get caught up in events that lead to the first opium war.', 'wikipedia_sentence')"
"('due to the gale force winds brought by the storm, the fleet was forced to take measures for self-preservation considering the shallow water depth and were ordered to weigh anchor by the flagship santo cristo de san agustín y nuestra señora del rosario.', 'wikipedia_sentence')"
"('beta thalassemias are caused by mutations in the hbb gene on chromosome 11, inherited in an autosomal recessive fashion.', 'wikipedia_sentence')"
"('the drug has been known to cause flushing, headaches and hypotension.', 'wikipedia_sentence')"
"('through most of his life he had suffered from a weak heart which would eventually result in his death.', 'wikipedia_sentence')"
"('the turnover led to a touchdown and a 14–7 lead.', 'wikipedia_sentence')"
"('thirty deaths were attributed to the earthquake.', 'wikipedia_sentence')"
"('the roof though has caused problems and in june 2012 the bus station had to be closed for two weeks to allow contractors to replace much of the roof to fix leaks.', 'wikipedia_sentence')"
"('her murder goes unsolved for a decade until dna evidence leads to an arrest and conviction.', 'wikipedia_sentence')"
"('dyspareunia is most commonly caused by ""vulvar and vaginal atrophy.""', 'wikipedia_sentence')"
"('the popularity of some tracks also led to groups considering releasing more material for the game.', 'wikipedia_sentence')"


We consider as incorrect:
- 28, 34, 72, 98

$\rightarrow$ estimated precision of $\frac{96}{100}=0.96$

### Support 3-21

In [23]:
random_samples(graph_type=('wikipedia_sentence', 'range(3, 38)'))

"('heywood was found dead in a hotel room in chongqing, and initial official reports (which have subsequently been challenged) attributed his death to alcohol poisoning.', 'wikipedia_sentence')","(""hahn's father has since confirmed that the cause of death was alcohol poisoning."", 'wikipedia_sentence')","('in 1971, a 17-year-old potter stewart high school student was sexually assaulted at a party and later died in sandy springs, georgia, although it is unclear whether her death was the result of alcohol poisoning or murder.', 'wikipedia_sentence')"
"('bulimia nervosa symptoms include heart problems like an irregular heartbeat that can lead to heart failure and death may occur.', 'wikipedia_sentence')","(""thompson's reported cause of death was an irregular heartbeat, the same cause of death as lynn's previous husband."", 'wikipedia_sentence')","('his death was caused by an irregular heartbeat according to kern county coroner officials.', 'wikipedia_sentence')"
"('the damage caused by floods is often recounted in historical documentation.', 'wikipedia_sentence')","('the nuns of the monastery responded by elevating the floor levels of the monastic buildings to reduce the damage caused by the floods.', 'wikipedia_sentence')","('in july 2015, the parliament approved aid worth 66.5 million eur to compensate the damage caused by the floods of 2014 in romania, italy and bulgaria in response to a report coordinated by mureșan.', 'wikipedia_sentence')"
"('the death was caused by cerebral haemorrhage.', 'wikipedia_sentence')","('however, in april 1906, he suffered a paralytic stroke and cerebral haemorrhage, which led to his death two months later.', 'wikipedia_sentence')","(""stephens' heavy drinking had aggravated his high blood pressure, and he died on 26 january 1959; the official cause of death was a cerebral haemorrhage brought about by chronic hypertension."", 'wikipedia_sentence')"
"('the storm caused power outages, and one house sustained fire damage due to candles and kerosene lamps.', 'wikipedia_sentence')","('the storm caused power outages on the island, and three people died due to using a generator inside their home during the storm.', 'wikipedia_sentence')","('the storm caused nearly 35,000 or more power outages in the south, with at least five fatalities confirmed.', 'wikipedia_sentence')"
"('when clive found matthew dead he felt responsible, unaware that the cause of death was carbon monoxide poisoning.', 'wikipedia_sentence')","('the cause of death was later confirmed to be carbon monoxide poisoning, due to obstructed ventilation from a gas-fired water boiler.', 'wikipedia_sentence')","('carbon monoxide poisoning was the main cause of death.', 'wikipedia_sentence')"
"('the earthquake caused extensive damage in a zone extending approximately from copiapó to coquimbo.', 'wikipedia_sentence')","('the earthquake also caused extensive damage to the north court area of scotia, with numerous homes damaged and gas leakages from a damaged gas line.', 'wikipedia_sentence')","('september 9 – an earthquake in rome causes extensive damage, including the collapse of the southern exterior facade of the colosseum.', 'wikipedia_sentence')"
"('postwar examination of japanese records indicates that the explosion, which resulted in extensive damage, was caused by a kaiten, a one-man suicide japanese submarine.', 'wikipedia_sentence')","('a further petition was sent to the board of ordnance in 1767 following an explosion which caused extensive damage.', 'wikipedia_sentence')","('however, due to the extensive damage caused by the explosion, the philippine national police ruled this out and considered the cause to be a bomb.', 'wikipedia_sentence')"
"('for instance, a cyclone that hit in 1970 caused 300,000 deaths and 86.4\xa0million usd of property damage.', 'wikipedia_sentence')","('as to germany, nine deaths caused by the cyclone have been reported.', 'wikipedia_sentence')","('save the children estimated the number of deaths associated w

We consider as incorrect:
- 24, 74

$\rightarrow$ estimated precision of $\frac{98}{100}=0.98$

## Manual Evaluation of Wikipedia-list Graph

In [24]:
random_samples(graph_type=('wikipedia_list',))

Cryptosporidiosis: https://en.wikipedia.org/w/index.php?&oldid=857162536 (wikipedia_list)
Gastroenterocolitis: https://en.wikipedia.org/w/index.php?&oldid=758295281 (wikipedia_list)
Fire breather's pneumonia: https://en.wikipedia.org/w/index.php?&oldid=860741927 (wikipedia_list)
Colpocephaly: https://en.wikipedia.org/w/index.php?&oldid=815912122 (wikipedia_list)
Mastocytosis: https://en.wikipedia.org/w/index.php?&oldid=855771465 (wikipedia_list)
Dilated cardiomyopathy: https://en.wikipedia.org/w/index.php?&oldid=844714059 (wikipedia_list)
Pancytopenia: https://en.wikipedia.org/w/index.php?&oldid=860991385 (wikipedia_list)
Hypotension: https://en.wikipedia.org/w/index.php?&oldid=861203011 (wikipedia_list)
Asystole: https://en.wikipedia.org/w/index.php?&oldid=853598448 (wikipedia_list)
Central nervous system cyst: https://en.wikipedia.org/w/index.php?&oldid=844403728 (wikipedia_list)
Onycholysis: https://en.wikipedia.org/w/index.php?&oldid=860519789 (wikipedia_list)


We consider as incorrect:
- 1, 51, 73

$\rightarrow$ estimated precision of $\frac{97}{100}=0.97$

## Manual Evaluation of Wikipedia-infobox Graph

In [25]:
random_samples(graph_type=('wikipedia_infobox',))

Lipoma: https://en.wikipedia.org/w/index.php?&oldid=855838460 (wikipedia_infobox)
Epilepsy: https://en.wikipedia.org/w/index.php?&oldid=861334494 (wikipedia_infobox)
2015 Peruvian protests against Las Bambas mining project: https://en.wikipedia.org/w/index.php?&oldid=846877894 (wikipedia_infobox)
Hyperemesis gravidarum: https://en.wikipedia.org/w/index.php?&oldid=860773220 (wikipedia_infobox)
Thirty Meter Telescope protests: https://en.wikipedia.org/w/index.php?&oldid=835207423 (wikipedia_infobox)
Intracerebral hemorrhage: https://en.wikipedia.org/w/index.php?&oldid=861487611 (wikipedia_infobox)
Onychomycosis: https://en.wikipedia.org/w/index.php?&oldid=861258650 (wikipedia_infobox)
Alzheimer's disease: https://en.wikipedia.org/w/index.php?&oldid=861534048 (wikipedia_infobox)
1989 Jordanian protests: https://en.wikipedia.org/w/index.php?&oldid=852327602 (wikipedia_infobox)
Meningitis: https://en.wikipedia.org/w/index.php?&oldid=860134524 (wikipedia_infobox)
Death of Savita Halappanavar: https://en.wikipedia.org/w/index.php?&oldid=861999092 (wikipedia_infobox)


We consider as incorrect:
 - 41, 42, 46, 54
 
$\rightarrow$ estimated precision of $\frac{96}{100}=0.96$